In [84]:
#Import neceassary libraries 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import BytesIO
from datetime import datetime

# Step 1 : Data Retrieval

In [ ]:
#Retrieve Data

def retrieve_fuelcheck_monthly_data():
    print("Retrieving NSW FuelCheck monthly data from Jan 2024 – Mar 2025")

    base_url = "https://data.nsw.gov.au/data/dataset/fuel-check"
    html_response = requests.get(base_url)

    if(html_response.status_code != 200):
        print("Failed to access page:", html_response.status_code)
        return pd.DataFrame()
    
    soup = BeautifulSoup(html_response.text, "html.parser")

    allowed_extensions = ['.xlsx', '.xls', '.csv']
    long_months = ['january', 'february', 'march', 'april', 'may', 'june',
                   'july', 'august', 'september', 'october', 'november', 'december']
    short_months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun',
                    'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

    target_patterns = [m + "2024" for m in long_months + short_months]
    target_patterns += [m + "2025" for m in long_months[:3] + short_months[:3]]
    target_patterns += [m + "25" for m in long_months[:3] + short_months[:3]]

    download_links = []
    for tag in soup.select("a[href$='.xls'], a[href$='.xlsx'], a[href$='.csv']"):
        href = tag['href'].lower()
        clean_href = href.replace('-', '').replace('_', '')
        if any(pattern in clean_href for pattern in target_patterns):
            download_links.append(tag['href'])

    print(f"Found {len(download_links)} monthly files.")

    monthly_dataframes = []
    for file_link in download_links:
        try:
            print(f"Downloading: {file_link}")
            response = requests.get(file_link)
            if(file_link.endswith(('.xls', '.xlsx'))):
                df_month = pd.read_excel(BytesIO(response.content))
            elif(file_link.endswith('.csv')):
                df_month = pd.read_csv(BytesIO(response.content))
            else:
                continue
            df_month['source_file'] = file_link
            monthly_dataframes.append(df_month)
        except Exception as e:
            print(f"Failed to load {file_link}: {e}")

    if(monthly_dataframes):
        combined_df = pd.concat(monthly_dataframes, ignore_index=True)
        print(f"Combined dataset shape: {combined_df.shape}")
        return combined_df
    else:
        print("No data loaded.")
        return pd.DataFrame()

In [95]:
def test_retrieve_fuelcheck_monthly_data(fuelcheck_raw_data):
    #Total number of rows and columns
    print("\nDataset shape (rows, columns):")
    print(fuelcheck_raw_data.shape)

    #Count missing (null) values in each column
    print("\nNull values per column:")
    print(fuelcheck_raw_data.isnull().sum())

    #First 10 rows of the dataset
    print("\n First 10 rows of the dataset:")
    display(fuelcheck_raw_data.head(10))

    #Check the datatypes of type columns 
    fuelcheck_raw_data.dtypes

    #Check dates
    print(fuelcheck_raw_data['PriceUpdatedDate'].astype(str).unique()[:5])


# Step 2 : Data Cleaning

In [ ]:
def data_cleaning(fuelcheck_raw_data):
    #Drop fully empty rows
    print("Rows before dropping empty rows:", len(fuelcheck_raw_data))
    fuelcheck_raw_data.dropna(how='all', inplace=True)
    print("Rows after dropping empty rows:", len(fuelcheck_raw_data))

    # Drop duplicate rows  
    print("Rows before dropping duplicates:", len(fuelcheck_raw_data))
    fuelcheck_raw_data.drop_duplicates(inplace=True)
    print("Rows after dropping duplicates:", len(fuelcheck_raw_data))

    # Strip leading/trailing spaces in all text columns
    str_columns = fuelcheck_raw_data.select_dtypes(include='object').columns

    print("Stripping whitespace from the following string columns:")
    print(str_columns.tolist())

    fuelcheck_raw_data[str_columns] = fuelcheck_raw_data[str_columns].apply(lambda col: col.str.strip())

    print("Whitespace removed.")

    # Issue with PriceUpdatedDate

    # Null/NaN values
    null_count = fuelcheck_raw_data['PriceUpdatedDate'].isnull().sum()

    # Blank/empty strings
    blank_count = fuelcheck_raw_data['PriceUpdatedDate'].astype(str).str.strip().eq('').sum()

    # Common invalid values
    invalid_values = ['--', '-', 'null', 'n/a', 'na', '0', 0]
    invalid_count = fuelcheck_raw_data['PriceUpdatedDate'].astype(str).str.lower().isin(invalid_values).sum()

    # Total bad values
    total_bad = null_count + blank_count + invalid_count

    print(f"PriceUpdatedDate column quality check:")
    print(f"Null values: {null_count}")
    print(f"Blank strings: {blank_count}")
    print(f"Common invalid entries: {invalid_count}")
    print(f"Total problematic entries: {total_bad}")

    # Sample rows with blank or invalid PriceUpdatedDate
    bad_rows = fuelcheck_raw_data[
    fuelcheck_raw_data['PriceUpdatedDate'].isnull() |
    fuelcheck_raw_data['PriceUpdatedDate'].astype(str).str.strip().isin(['', '--', '-', 'null', 'n/a', 'na', '0'])
    ]

    display(bad_rows[['PriceUpdatedDate', 'source_file']].head(10))

    # Convert Price column to numeric and remove outliers
    if('Price' in fuelcheck_raw_data.columns):
        print("Filtering out prices outside 50–300 cents range")
        before_rows = len(fuelcheck_raw_data)
        fuelcheck_raw_data = fuelcheck_raw_data[(fuelcheck_raw_data['Price'] >= 50) & (fuelcheck_raw_data['Price'] <= 300)]
        after_rows = len(fuelcheck_raw_data)
        print(f"Filtered out {before_rows - after_rows} rows with invalid prices.")
    else:
        print("'Price' column not found in dataset.")

    # Extracting the months from source link to apply default date to null values
    def infer_date_from_filename(filename):
        month_map = {
            'jan': 1, 'january': 1,
            'feb': 2, 'february': 2,
            'mar': 3, 'march': 3,
            'apr': 4, 'april': 4,
            'may': 5,
            'jun': 6, 'june': 6,
            'jul': 7, 'july': 7,
            'aug': 8, 'august': 8,
            'sep': 9, 'september': 9,
            'oct': 10, 'october': 10,
            'nov': 11, 'november': 11,
            'dec': 12, 'december': 12
        }

        filename = filename.lower().replace('-', '').replace('_', '')
        for key, month in month_map.items():
            if key in filename:
                if '2024' in filename:
                    return datetime(2024, month, 1)
                elif '2025' in filename or '25' in filename:
                    return datetime(2025, month, 1)
        return pd.NaT
    
    # Fill missing PriceUpdatedDate using source_file name
    print("Remaining nulls in 'PriceUpdatedDate':", fuelcheck_raw_data['PriceUpdatedDate'].isnull().sum())
    if('PriceUpdatedDate' in fuelcheck_raw_data.columns and 'source_file' in fuelcheck_raw_data.columns):
        null_count = fuelcheck_raw_data['PriceUpdatedDate'].isnull().sum()
        print(f"Filling {null_count} missing dates using file name")

        fuelcheck_raw_data['PriceUpdatedDate'] = fuelcheck_raw_data.apply(
            lambda row: row['PriceUpdatedDate'] if pd.notnull(row['PriceUpdatedDate'])
            else infer_date_from_filename(row['source_file']),
            axis=1
        )

        print("Remaining nulls in 'PriceUpdatedDate':", fuelcheck_raw_data['PriceUpdatedDate'].isnull().sum())
    else:
        print("Required columns for date fill not found")

    # Convert PriceUpdatedDate to datetime format
    if('PriceUpdatedDate' in fuelcheck_raw_data.columns):
        print("Converting 'PriceUpdatedDate' to datetime")
        fuelcheck_raw_data['PriceUpdatedDate'] = pd.to_datetime(
            fuelcheck_raw_data['PriceUpdatedDate'], errors='coerce'
        )
        print("Nulls in 'PriceUpdatedDate' after conversion:", fuelcheck_raw_data['PriceUpdatedDate'].isnull().sum())
    else:
        print("'PriceUpdatedDate' column not found in dataset")

    fuelcheck_raw_data['Date'] = fuelcheck_raw_data['PriceUpdatedDate'].dt.date
    fuelcheck_raw_data['Time'] = fuelcheck_raw_data['PriceUpdatedDate'].dt.time

    # Drop rows missing any of the key fields
    required_fields = ['ServiceStationName', 'PriceUpdatedDate', 'Price']

    print("Dropping rows with missing values in required fields:", required_fields)
    before_drop = len(fuelcheck_raw_data)

    fuelcheck_raw_data.dropna(subset=[col for col in required_fields if col in fuelcheck_raw_data.columns], inplace=True)

    after_drop = len(fuelcheck_raw_data)
    print(f"Dropped {before_drop - after_drop} rows. Final dataset shape: {fuelcheck_raw_data.shape}")

    #Shape of the dataset
    print("Shape (rows, columns):", fuelcheck_raw_data.shape)

    # Null check
    print("\nRemaining nulls per column:")
    print(fuelcheck_raw_data.isnull().sum())
    # Data types
    print("\nColumn data types:")
    print(fuelcheck_raw_data.dtypes)

    # Price range summary
    print("\nPrice column summary:")
    print(fuelcheck_raw_data['Price'].describe())

    # Unique fuel types
    if('FuelCode' in fuelcheck_raw_data.columns):
        print("\nUnique Fuel Types:")
        print(fuelcheck_raw_data['FuelCode'].value_counts())

    # Random sample
    print("\nSample rows:")
    display(fuelcheck_raw_data.sample(5, random_state=42))

In [101]:
# Convert cleaned data to CSV
def convert_cleaned_data_to_csv(fuelcheck_raw_data):    
    output_file = "cleaned_fuelcheck_data.csv"
    fuelcheck_raw_data.to_csv(output_file, index=False)
    print(f"Converted Cleaned data saved to {output_file}")

In [103]:
# Main Function -
def main():
    #Step 1: Retrieving the data
    fuelcheck_raw_data = retrieve_fuelcheck_monthly_data() #Calling Retrivel Function 
    print("Raw Data", fuelcheck_raw_data)
    test_retrieve_fuelcheck_monthly_data(fuelcheck_raw_data) #Calling Test Function

    #Step 2: Data Cleaning
    data_cleaning(fuelcheck_raw_data) #Calling Data Cleaning Function

    #Save the cleaned data to CSV
    convert_cleaned_data_to_csv(fuelcheck_raw_data) #Calling Convert Function

main() #Calling Main Function

Retrieving NSW FuelCheck monthly data from Jan 2024 – Mar 2025
Found 15 monthly files.
Downloading: https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/2d785043-38eb-4eeb-b992-89f1af8d91e1/download/fuelcheck_pricehistory_jan2024.xlsx
Downloading: https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/404dcdf6-bbde-4ccf-869e-259bcc408ce5/download/fuelcheck_pricehistory_feb2024.xlsx
Downloading: https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/8c461685-3ed2-47e0-b9e6-9737c58a21bf/download/fuelcheck_pricehistory_mar2024.xlsx
Downloading: https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/56a4fdb5-0789-4a2f-a89f-18e0159e210f/download/fuelcheck_pricehistory_apr2024.xlsx
Downloading: https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/f70eb8e4-7291-47c8-874c-daea7eda0df1/download/fuelcheck_pricehistory_may2024.xlsx
Downloading: https://da

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price,source_file
0,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,U91,2024-01-01 00:44:05,168.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
1,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,P98,2024-01-01 00:44:05,189.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
2,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,PDL,2024-01-01 00:44:05,177.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
3,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,E10,2024-01-01 00:44:11,164.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
4,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,P98,2024-01-01 00:44:11,188.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
5,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,PDL,2024-01-01 00:44:11,176.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
6,Costco Lake Macquarie (Members only),"1 CRESSY RD, BOOLAROO NSW 2284",BOOLAROO,2284,Costco,E10,2024-01-01 00:44:17,166.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
7,Costco Lake Macquarie (Members only),"1 CRESSY RD, BOOLAROO NSW 2284",BOOLAROO,2284,Costco,P98,2024-01-01 00:44:17,188.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
8,Costco Lake Macquarie (Members only),"1 CRESSY RD, BOOLAROO NSW 2284",BOOLAROO,2284,Costco,PDL,2024-01-01 00:44:17,178.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...
9,Costco Marsden Park (Members only),"10 Langford Drive, MARSDEN PARK NSW 2765",MARSDEN PARK,2765,Costco,E10,2024-01-01 00:44:22,168.7,https://data.nsw.gov.au/data/dataset/a97a46fc-...


['2024-01-01 00:44:05' '2024-01-01 00:44:11' '2024-01-01 00:44:17'
 '2024-01-01 00:44:22' '2024-01-01 00:58:10']
Rows before dropping empty rows: 1128302
Rows after dropping empty rows: 1128302
Rows before dropping duplicates: 1128302
Rows after dropping duplicates: 1128287
Stripping whitespace from the following string columns:
['ServiceStationName', 'Address', 'Suburb', 'Brand', 'FuelCode', 'PriceUpdatedDate', 'source_file']
Whitespace removed.
PriceUpdatedDate column quality check:
Null values: 836614
Blank strings: 0
Common invalid entries: 0
Total problematic entries: 836614


,PriceUpdatedDate,source_file
0,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
1,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
2,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
3,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
4,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
5,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
6,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
7,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
8,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...
9,NaN,https://data.nsw.gov.au/data/dataset/a97a46fc-...


Filtering out prices outside 50–300 cents range
Filtered out 0 rows with invalid prices.
Remaining nulls in 'PriceUpdatedDate': 836614
Filling 836614 missing dates using file name
Remaining nulls in 'PriceUpdatedDate': 0
Converting 'PriceUpdatedDate' to datetime
Nulls in 'PriceUpdatedDate' after conversion: 0
Dropping rows with missing values in required fields: ['ServiceStationName', 'PriceUpdatedDate', 'Price']
Dropped 0 rows. Final dataset shape: (1128287, 9)
Shape (rows, columns): (1128287, 9)

Remaining nulls per column:
ServiceStationName    0
Address               0
Suburb                0
Postcode              0
Brand                 0
FuelCode              0
PriceUpdatedDate      0
Price                 0
source_file           0
dtype: int64

Column data types:
ServiceStationName            object
Address                       object
Suburb                        object
Postcode                       int64
Brand                         object
FuelCode                      obje

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price,source_file
890685,BP Potts Hill T/S,"155-157 Rockwood Road, Yagoona NSW 2199",Yagoona,2199,BP,P95,2024-12-01 00:00:00,206.9,https://data.nsw.gov.au/data/dataset/a97a46fc-...
1027559,Liberty Foodmart,"54 Ballina Rd, LISMORE NSW 2480",LISMORE,2480,Liberty,DL,2025-02-10 13:53:30,189.9,https://data.nsw.gov.au/data/dataset/a97a46fc-...
1002455,Budget Campsie,"403 Canterbury Road, Campsie NSW 2194",Campsie,2194,Budget,E10,2025-02-03 13:06:58,189.9,https://data.nsw.gov.au/data/dataset/a97a46fc-...
394745,Shell Reddy Express Hurstville,"396-404 Forest Road, Hurstville NSW 2220",Hurstville,2220,Reddy Express,P95,2024-06-01 00:00:00,234.9,https://data.nsw.gov.au/data/dataset/a97a46fc-...
628432,Ultra Manly,"209 Pittwater Road, Manly NSW 2095",Manly,2095,Ultra Petroleum,U91,2024-09-01 00:00:00,195.9,https://data.nsw.gov.au/data/dataset/a97a46fc-...


Converted Cleaned data saved to cleaned_fuelcheck_data.csv
